Steps:
1. Finalise data sets (be brutal, identify roots and stems; address missing values, model missing value evaluate to mean)
2. Model linear regression statistics (feature importances; chicken feed/auto)
3. Prediction: random forest
4. data visualisation (pairplots)

In [1]:
import pandas as pd

### Covid 19 Cases by County (USA Facts/CDC)

For most states, USAFacts directly collects the daily county-level cumulative totals of positive cases and deaths from a table, dashboard, or PDF on the state public health website. This data is compiled either through scraping or manual entry. The underlying data is available for download below the US county map and has helped government agencies like the Centers for Disease Control and Prevention in its nationwide efforts.

REFERENCES:
1. https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/

In [2]:
covid_cases = pd.read_csv("data/covid_confirmed_usafacts_200803.csv")

In [3]:
covid_cases.head()

countyFIPS            County Name State  stateFIPS  1/22/20  1/23/20  \
0           0  Statewide Unallocated    AL          1        0        0   
1        1001         Autauga County    AL          1        0        0   
2        1003         Baldwin County    AL          1        0        0   
3        1005         Barbour County    AL          1        0        0   
4        1007            Bibb County    AL          1        0        0   

   1/24/20  1/25/20  1/26/20  1/27/20  ...  7/23/20  7/24/20  7/25/20  \
0        0        0        0        0  ...        0        0        0   
1        0        0        0        0  ...      905      921      932   
2        0        0        0        0  ...     2461     2513     2662   
3        0        0        0        0  ...      534      539      552   
4        0        0        0        0  ...      289      303      318   

   7/26/20  7/27/20  7/28/20  7/29/20  7/30/20  7/31/20  8/1/20  
0        0        0        0        0        0        0       0  
1      942      965      974      974     1002     1015    1030  
2     2708     2770     2835     2835     3028     3101    3142  
3      562      569      575      575      585      598     602  
4      324      334      337      338      352      363     368  

[5 rows x 197 columns]

In [4]:
covid_cases_dropped = covid_cases.drop(columns=['8/1/20'])

In [5]:
covid_cases_dropped_only = covid_cases_dropped.iloc[:,-192:]

In [6]:
covid_cases_total = covid_cases_dropped['Total Cases']= covid_cases_dropped.iloc[:, -192:].sum(axis=1)

In [7]:
covid_cases_filter = covid_cases_dropped.loc[:,["countyFIPS", "County Name", "State", "stateFIPS", "Total Cases"]]
covid_cases_filter["countyFIPS"] = covid_cases_filter["countyFIPS"].astype(str)
print(covid_cases_filter.dtypes)

countyFIPS     object
County Name    object
State          object
stateFIPS       int64
Total Cases     int64
dtype: object


In [8]:
covid_cases_filter['countyFIPS_2d'] = covid_cases_filter['countyFIPS'].str[-3:]
covid_cases_filter['countyFIPS'] = covid_cases_filter['countyFIPS'].astype(str).str.zfill(5)
covid_cases_filter['countyFIPS_2d'] = covid_cases_filter['countyFIPS_2d'].astype(str).str.zfill(3)
covid_cases_filter['stateFIPS'] = covid_cases_filter['stateFIPS'].astype(str).str.zfill(2)
covid_cases_filter = covid_cases_filter.loc[:,["countyFIPS", "stateFIPS", "countyFIPS_2d", "County Name", "State", "Total Cases"]]

In [9]:
covid_cases_clean = covid_cases_filter.copy()

In [10]:
covid_cases_clean = covid_cases_clean.loc[covid_cases_clean['County Name'] != "Statewide Unallocated"]
covid_cases_clean

countyFIPS stateFIPS countyFIPS_2d        County Name State  Total Cases
1         01001        01           001     Autauga County    AL        39746
2         01003        01           003     Baldwin County    AL        76970
3         01005        01           005     Barbour County    AL        24625
4         01007        01           007        Bibb County    AL        13636
5         01009        01           009      Blount County    AL        19311
...         ...       ...           ...                ...   ...          ...
3190      56037        56           037  Sweetwater County    WY         7361
3191      56039        56           039       Teton County    WY        13823
3192      56041        56           041       Uinta County    WY         9737
3193      56043        56           043    Washakie County    WY         3104
3194      56045        56           045      Weston County    WY          130

[3145 rows x 6 columns]

In [11]:
test_cases = covid_cases_clean.loc[(covid_cases_clean["countyFIPS"] == "51560")]
test_cases

Empty DataFrame
Columns: [countyFIPS, stateFIPS, countyFIPS_2d, County Name, State, Total Cases]
Index: []

In [12]:
null_data_cases = covid_cases_clean[covid_cases_clean.isnull().any(axis=1)]
null_data_cases

Empty DataFrame
Columns: [countyFIPS, stateFIPS, countyFIPS_2d, County Name, State, Total Cases]
Index: []

### Covid 19 Deaths by County (USA Facts/CDC)

For most states, USAFacts directly collects the daily county-level cumulative totals of positive cases and deaths from a table, dashboard, or PDF on the state public health website. This data is compiled either through scraping or manual entry. The underlying data is available for download below the US county map and has helped government agencies like the Centers for Disease Control and Prevention in its nationwide efforts.

REFERENCES:
1. https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/

In [13]:
covid_deaths = pd.read_csv("data/covid_deaths_usafacts_200803.csv")

In [14]:
covid_deaths_dropped = covid_deaths.drop(columns=['8/1/20'])

In [15]:
covid_deaths_total = covid_deaths_dropped['Total Deaths']= covid_deaths_dropped.iloc[:, -192:].sum(axis=1)

In [16]:
covid_deaths_filter = covid_deaths_dropped.loc[:,["countyFIPS", "County Name", "State", "stateFIPS", "Total Deaths"]]

In [17]:
covid_deaths_filter = covid_deaths_dropped.loc[:,["countyFIPS", "County Name", "State", "stateFIPS", "Total Deaths"]]
covid_deaths_filter["countyFIPS"] = covid_deaths_filter["countyFIPS"].astype(str)
print(covid_deaths_filter.dtypes)

countyFIPS      object
County Name     object
State           object
stateFIPS        int64
Total Deaths     int64
dtype: object


In [18]:
covid_deaths_filter['countyFIPS_2d'] = covid_deaths_filter['countyFIPS'].str[-3:]
covid_deaths_filter['countyFIPS'] = covid_deaths_filter['countyFIPS'].astype(str).str.zfill(5)
covid_deaths_filter['countyFIPS_2d'] = covid_deaths_filter['countyFIPS_2d'].astype(str).str.zfill(3)
covid_deaths_filter['stateFIPS'] = covid_deaths_filter['stateFIPS'].astype(str).str.zfill(2)
covid_deaths_filter = covid_deaths_filter.loc[:,["countyFIPS", "stateFIPS", "countyFIPS_2d", "County Name", "State", "Total Deaths"]]
covid_deaths_filter

countyFIPS stateFIPS countyFIPS_2d            County Name State  \
0         00000        01           000  Statewide Unallocated    AL   
1         01001        01           001         Autauga County    AL   
2         01003        01           003         Baldwin County    AL   
3         01005        01           005         Barbour County    AL   
4         01007        01           007            Bibb County    AL   
...         ...       ...           ...                    ...   ...   
3190      56037        56           037      Sweetwater County    WY   
3191      56039        56           039           Teton County    WY   
3192      56041        56           041           Uinta County    WY   
3193      56043        56           043        Washakie County    WY   
3194      56045        56           045          Weston County    WY   

      Total Deaths  
0                0  
1              909  
2              958  
3              155  
4              103  
...            ...  
3190            34  
3191           101  
3192             0  
3193           291  
3194             0  

[3195 rows x 6 columns]

In [19]:
covid_deaths_clean = covid_deaths_filter.copy()
covid_deaths_clean = covid_deaths_clean.loc[covid_deaths_clean['County Name'] != "Statewide Unallocated"]

In [20]:
covid_deaths_clean.describe()
test_deaths_1 = covid_deaths_clean.loc[(covid_deaths_clean["countyFIPS"] == "15005")]
test_deaths_1

countyFIPS stateFIPS countyFIPS_2d     County Name State  Total Deaths
561      15005        15           005  Kalawao County    HI             0

In [21]:
null_data_deaths = covid_deaths_clean[covid_deaths_clean.isnull().any(axis=1)]
null_data_deaths

Empty DataFrame
Columns: [countyFIPS, stateFIPS, countyFIPS_2d, County Name, State, Total Deaths]
Index: []

In [22]:
covid_deaths_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3146 entries, 1 to 3194
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   countyFIPS     3146 non-null   object
 1   stateFIPS      3146 non-null   object
 2   countyFIPS_2d  3146 non-null   object
 3   County Name    3146 non-null   object
 4   State          3146 non-null   object
 5   Total Deaths   3146 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 172.0+ KB


### Per capital incidence of poverty by U.S county (U.S Census)

The poverty universe is made up of persons for whom the Census Bureau can determine poverty status (either "in poverty" or "not in poverty").

REFERENCES:
1. SAIPE Model Input Data: https://www.census.gov/data/datasets/time-series/demo/saipe/model-tables.html

In [23]:
poverty = pd.read_csv("data/allpovu.csv")
poverty_all_ages = poverty.loc[:,["State FIPS code", "County FIPS code", "Name", "State Postal Code", "Poverty Universe, All Ages"]]
poverty_all_ages.rename(columns={'State FIPS code': 'stateFIPS', 'County FIPS code': 'countyFIPS_2d'}, inplace=True)
poverty_all_ages

stateFIPS  countyFIPS_2d               Name State Postal Code  \
0             0              0      United States                US   
1             1              0            Alabama                AL   
2             1              1     Autauga County                AL   
3             1              3     Baldwin County                AL   
4             1              5     Barbour County                AL   
...         ...            ...                ...               ...   
3196         56             37  Sweetwater County                WY   
3197         56             39       Teton County                WY   
3198         56             41       Uinta County                WY   
3199         56             43    Washakie County                WY   
3200         56             45      Weston County                WY   

      Poverty Universe, All Ages  
0                    319184033.0  
1                      4763811.0  
2                        55073.0  
3                       215255.0  
4                        21979.0  
...                          ...  
3196                     42205.0  
3197                     22888.0  
3198                     20135.0  
3199                      7735.0  
3200                      6595.0  

[3201 rows x 5 columns]

In [24]:
poverty_all_ages.rename(columns={'Name': 'County Name', 'State Postal Code': 'State'}, inplace=True)
poverty_clean = poverty_all_ages.copy()
poverty_clean['countyFIPS_2d'] = poverty_clean['countyFIPS_2d'].astype(str).str.zfill(3)
poverty_clean['stateFIPS'] = poverty_clean['stateFIPS'].astype(str).str.zfill(2)
poverty_clean["countyFIPS"] = poverty_clean["stateFIPS"] + poverty_clean["countyFIPS_2d"]

In [25]:
poverty_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3201 entries, 0 to 3200
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   stateFIPS                   3201 non-null   object 
 1   countyFIPS_2d               3201 non-null   object 
 2   County Name                 3201 non-null   object 
 3   State                       3201 non-null   object 
 4   Poverty Universe, All Ages  3193 non-null   float64
 5   countyFIPS                  3201 non-null   object 
dtypes: float64(1), object(5)
memory usage: 150.2+ KB


In [26]:
poverty_clean = poverty_clean.loc[poverty_clean['countyFIPS_2d'] != 0]

In [27]:
poverty_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3201 entries, 0 to 3200
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   stateFIPS                   3201 non-null   object 
 1   countyFIPS_2d               3201 non-null   object 
 2   County Name                 3201 non-null   object 
 3   State                       3201 non-null   object 
 4   Poverty Universe, All Ages  3193 non-null   float64
 5   countyFIPS                  3201 non-null   object 
dtypes: float64(1), object(5)
memory usage: 175.1+ KB


In [28]:
null_data_pov = poverty_clean[poverty_clean.isnull().any(axis=1)]
null_data_pov

stateFIPS countyFIPS_2d                                  County Name  \
92          02           201  Prince of Wales-Outer Ketchikan Census Area   
95          02           232            Skagway-Hoonah-Angoon Census Area   
98          02           270                     Wade Hampton Census Area   
100         02           280              Wrangell-Petersburg Census Area   
565         15           005                               Kalawao County   
2465        46           113                               Shannon County   
2969        51           515                                 Bedford city   
2974        51           560                                Clifton Forge   

     State  Poverty Universe, All Ages countyFIPS  
92      AK                         NaN      02201  
95      AK                         NaN      02232  
98      AK                         NaN      02270  
100     AK                         NaN      02280  
565     HI                         NaN      15005  
2465    SD                         NaN      46113  
2969    VA                         NaN      51515  
2974    VA                         NaN      51560

In [29]:
poverty_clean['Poverty Universe, All Ages'] = poverty_clean['Poverty Universe, All Ages'].fillna((poverty_clean['Poverty Universe, All Ages'].mean()))

In [30]:
poverty_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3201 entries, 0 to 3200
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   stateFIPS                   3201 non-null   object 
 1   countyFIPS_2d               3201 non-null   object 
 2   County Name                 3201 non-null   object 
 3   State                       3201 non-null   object 
 4   Poverty Universe, All Ages  3201 non-null   float64
 5   countyFIPS                  3201 non-null   object 
dtypes: float64(1), object(5)
memory usage: 175.1+ KB


In [31]:
test_pov = poverty_clean.loc[(poverty_clean["countyFIPS"] == "46102")]
test_pov

stateFIPS countyFIPS_2d           County Name State  \
2459        46           102  Oglala Lakota County    SD   

      Poverty Universe, All Ages countyFIPS  
2459                     14235.0      46102

### County Population by Racial/Ethnic Characteristics 2010-2019 (U.S. Census Bureau)

METHODOLOGY FOR THE UNITED STATES POPULATION ESTIMATES: VINTAGE 2019
Nation, States, Counties, and Puerto Rico – April 1, 2010 to July 1, 2019

Each year, the United States Census Bureau produces and publishes estimates of the population for the
nation, states, counties, state/county equivalents, and Puerto Rico.1 We estimate the resident population for
each year since the most recent decennial census by using measures of population change. The resident
population includes all people currently residing in the United States.

With each annual release of population estimates, the Population Estimates Program revises and updates the
entire time series of estimates from April 1, 2010 to July 1 of the current year, which we refer to as the
vintage year. We use the term “vintage” to denote an entire time series created with a consistent population
starting point and methodology. The release of a new vintage of estimates supersedes any previous series
and incorporates the most up-to-date input data and methodological improvements

REFERENCES:
1. Annual County Resident Population Estimates by Age, Sex, Race, and Hispanic Origin: April 1, 2010 to July 1, 2019 (https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-detail.html)
2. File Layout: https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2019/cc-est2019-alldata.pdf

In [32]:
race = pd.read_csv("data/cc-est2019-alldata.csv", encoding = "ISO-8859-1")

In [33]:
# race.columns.tolist()

# SELECTION - Z Value
# sum columns by race and gender 
# e.g. race["WA_MALE_TOTAL"] = race.loc[:, ["WA_MALE", "WAC_MALE"].sum()

# WA_MALE
# WAC_MALE

# WA_FEMALE
# WAC_FEMALE

# BA_MALE
# BAC_MALE

# BA_FEMALE
# BAC_FEMALE

# IA_MALE
# IAC_MALE

# IA_FEMALE
# IAC_FEMALE

# AA_MALE
# AAC_MALE 

# AA_FEMALE
# AAC_FEMALE

# NA_MALE
# NAC_MALE 

# NA_FEMALE
# NAC_FEMALE

# TOM_MALE
# TOM_FEMALE

race["WA_MALE_TOTAL"] = race.loc[:, ["WA_MALE", "WAC_MALE"]].sum(axis=1)
race["WA_FEMALE_TOTAL"] = race.loc[:, ["WA_FEMALE", "WAC_FEMALE"]].sum(axis=1)
race["BA_MALE_TOTAL"] = race.loc[:, ["BA_MALE", "BAC_MALE"]].sum(axis=1)
race["BA_FEMALE_TOTAL"] = race.loc[:, ["BA_FEMALE", "BAC_FEMALE"]].sum(axis=1)
race["IA_MALE_TOTAL"] = race.loc[:, ["IA_MALE", "IAC_MALE"]].sum(axis=1)
race["IA_FEMALE_TOTAL"] = race.loc[:, ["IA_FEMALE", "IAC_FEMALE"]].sum(axis=1)
race["AA_MALE_TOTAL"] = race.loc[:, ["AA_MALE", "AAC_MALE"]].sum(axis=1)
race["AA_FEMALE_TOTAL"] = race.loc[:, ["AA_FEMALE", "AAC_FEMALE"]].sum(axis=1)
race["NA_MALE_TOTAL"] = race.loc[:, ["NA_MALE", "NAC_MALE"]].sum(axis=1)
race["NA_FEMALE_TOTAL"] = race.loc[:, ["NA_FEMALE", "NAC_FEMALE"]].sum(axis=1)

In [34]:
race["YEAR"] = race["YEAR"].astype(int)
race

SUMLEV  STATE  COUNTY   STNAME         CTYNAME  YEAR  AGEGRP  TOT_POP  \
0           50      1       1  Alabama  Autauga County     1       0    54571   
1           50      1       1  Alabama  Autauga County     1       1     3579   
2           50      1       1  Alabama  Autauga County     1       2     3991   
3           50      1       1  Alabama  Autauga County     1       3     4290   
4           50      1       1  Alabama  Autauga County     1       4     4290   
...        ...    ...     ...      ...             ...   ...     ...      ...   
716371      50     56      45  Wyoming   Weston County    12      14      499   
716372      50     56      45  Wyoming   Weston County    12      15      352   
716373      50     56      45  Wyoming   Weston County    12      16      229   
716374      50     56      45  Wyoming   Weston County    12      17      198   
716375      50     56      45  Wyoming   Weston County    12      18      200   

        TOT_MALE  TOT_FEMALE  ...  WA_MALE_TOTAL  WA_FEMALE_TOTAL  \
0          26569       28002  ...          42928            44393   
1           1866        1713  ...           2890             2684   
2           2001        1990  ...           3091             3109   
3           2171        2119  ...           3352             3301   
4           2213        2077  ...           3292             3209   
...          ...         ...  ...            ...              ...   
716371       280         219  ...            514              409   
716372       180         172  ...            349              339   
716373       107         122  ...            212              240   
716374        82         116  ...            161              230   
716375        72         128  ...            141              251   

        BA_MALE_TOTAL  BA_FEMALE_TOTAL  IA_MALE_TOTAL  IA_FEMALE_TOTAL  \
0                9263            10436            396              453   
1                 767              679             28               21   
2                 824              777             41               27   
3                 884              842             44               39   
4                1027              868             35               27   
...               ...              ...            ...              ...   
716371              1                2              7                2   
716372              0                1              4                2   
716373              0                0              2                4   
716374              0                0              2                2   
716375              0                0              3                3   

        AA_MALE_TOTAL  AA_FEMALE_TOTAL  NA_MALE_TOTAL  NA_FEMALE_TOTAL  
0                 500              693             71               55  
1                  47               43              4                1  
2                  49               63              4                7  
3                  55               55              8                6  
4                  64               45             10                7  
...               ...              ...            ...              ...  
716371             38               25              0                0  
716372              7                2              0                0  
716373              0                0              0                0  
716374              1                0              0                0  
716375              0                2              0                0  

[716376 rows x 90 columns]

In [35]:
# Current is constrained to YEAR: 12 = 7/1/2019 & AGEGRP: 0 = Total

race_current = race.loc[(race['YEAR'] == 12) & (race['AGEGRP'] == 0)]
race_current.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3142 entries, 209 to 716357
Data columns (total 90 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   SUMLEV           3142 non-null   int64 
 1   STATE            3142 non-null   int64 
 2   COUNTY           3142 non-null   int64 
 3   STNAME           3142 non-null   object
 4   CTYNAME          3142 non-null   object
 5   YEAR             3142 non-null   int64 
 6   AGEGRP           3142 non-null   int64 
 7   TOT_POP          3142 non-null   int64 
 8   TOT_MALE         3142 non-null   int64 
 9   TOT_FEMALE       3142 non-null   int64 
 10  WA_MALE          3142 non-null   int64 
 11  WA_FEMALE        3142 non-null   int64 
 12  BA_MALE          3142 non-null   int64 
 13  BA_FEMALE        3142 non-null   int64 
 14  IA_MALE          3142 non-null   int64 
 15  IA_FEMALE        3142 non-null   int64 
 16  AA_MALE          3142 non-null   int64 
 17  AA_FEMALE        3142 non-nul

In [36]:
race_current.loc[:,["STATE", "COUNTY", "STNAME", "CTYNAME", "TOT_POP", "WA_MALE_TOTAL", "WA_FEMALE_TOTAL"
               , "BA_MALE_TOTAL", "BA_FEMALE_TOTAL", "IA_MALE_TOTAL", "IA_FEMALE_TOTAL"
               , "AA_MALE_TOTAL", "AA_FEMALE_TOTAL", "NA_MALE_TOTAL", "NA_FEMALE_TOTAL"]]

STATE  COUNTY   STNAME            CTYNAME  TOT_POP  WA_MALE_TOTAL  \
209         1       1  Alabama     Autauga County    55869          42250   
437         1       3  Alabama     Baldwin County   223234         191540   
665         1       5  Alabama     Barbour County    24686          12906   
893         1       7  Alabama        Bibb County    22394          17635   
1121        1       9  Alabama      Blount County    57826          54866   
...       ...     ...      ...                ...      ...            ...   
715445     56      37  Wyoming  Sweetwater County    42343          41325   
715673     56      39  Wyoming       Teton County    23464          23328   
715901     56      41  Wyoming       Uinta County    20226          19698   
716129     56      43  Wyoming    Washakie County     7805           7602   
716357     56      45  Wyoming      Weston County     6927           6867   

        WA_FEMALE_TOTAL  BA_MALE_TOTAL  BA_FEMALE_TOTAL  IA_MALE_TOTAL  \
209               43920          10751            12270            395   
437              202761          19832            21115           2721   
665               11608          12743            11280            285   
893               16976           5951             3719            159   
1121              56713           1174             1080            592   
...                 ...            ...              ...            ...   
715445            38927            828              640            881   
715673            21591            248              181            310   
715901            19209            199              186            391   
716129             7321             80               57            169   
716357             6211             87               60            211   

        IA_FEMALE_TOTAL  AA_MALE_TOTAL  AA_FEMALE_TOTAL  NA_MALE_TOTAL  \
209                 446            727              879             87   
437                2624           2337             3394            254   
665                 182            127              141             72   
893                 151             72               73             50   
1121                598            232              265            102   
...                 ...            ...              ...            ...   
715445              801            516              619             97   
715673              272            358              573             63   
715901              402            123              180             54   
716129              198             66              102             13   
716357              155             78              178              8   

        NA_FEMALE_TOTAL  
209                  75  
437                 267  
665                  41  
893                  16  
1121                 62  
...                 ...  
715445               99  
715673               43  
715901               44  
716129               13  
716357                2  

[3142 rows x 15 columns]

In [37]:
race_current.describe()

SUMLEV        STATE       COUNTY    YEAR  AGEGRP       TOT_POP  \
count  3142.0  3142.000000  3142.000000  3142.0  3142.0  3.142000e+03   
mean     50.0    30.280076   103.572884    12.0     0.0  1.044683e+05   
std       0.0    15.144339   107.704060     0.0     0.0  3.334567e+05   
min      50.0     1.000000     1.000000    12.0     0.0  8.600000e+01   
25%      50.0    18.000000    35.000000    12.0     0.0  1.090250e+04   
50%      50.0    29.000000    79.000000    12.0     0.0  2.572600e+04   
75%      50.0    45.000000   133.000000    12.0     0.0  6.807275e+04   
max      50.0    56.000000   840.000000    12.0     0.0  1.003911e+07   

           TOT_MALE    TOT_FEMALE       WA_MALE     WA_FEMALE  ...  \
count  3.142000e+03  3.142000e+03  3.142000e+03  3.142000e+03  ...   
mean   5.145045e+04  5.301789e+04  3.952652e+04  4.020683e+04  ...   
std    1.638677e+05  1.696276e+05  1.179341e+05  1.190460e+05  ...   
min    4.100000e+01  4.500000e+01  1.300000e+01  1.100000e+01  ...   
25%    5.459750e+03  5.407250e+03  4.544250e+03  4.461750e+03  ...   
50%    1.286900e+04  1.282850e+04  1.094450e+04  1.101950e+04  ...   
75%    3.415225e+04  3.451250e+04  2.910775e+04  2.975875e+04  ...   
max    4.949041e+06  5.090066e+06  3.552806e+06  3.545503e+06  ...   

       WA_MALE_TOTAL  WA_FEMALE_TOTAL  BA_MALE_TOTAL  BA_FEMALE_TOTAL  \
count   3.142000e+03     3.142000e+03   3.142000e+03     3.142000e+03   
mean    8.033665e+04     8.171576e+04   1.408558e+04     1.528941e+04   
std     2.401430e+05     2.424334e+05   5.646954e+04     6.512335e+04   
min     2.900000e+01     3.000000e+01   0.000000e+00     0.000000e+00   
25%     9.183000e+03     8.992500e+03   2.190000e+02     1.482500e+02   
50%     2.210150e+04     2.229450e+04   1.278500e+03     8.390000e+02   
75%     5.877825e+04     6.018950e+04   6.424000e+03     6.014000e+03   
max     7.240650e+06     7.229593e+06   1.140998e+06     1.362219e+06   

       IA_MALE_TOTAL  IA_FEMALE_TOTAL  AA_MALE_TOTAL  AA_FEMALE_TOTAL  \
count    3142.000000      3142.000000   3.142000e+03     3.142000e+03   
mean     1771.046467      1772.443030   6.459161e+03     7.024877e+03   
std      6577.875055      6677.061211   4.455279e+04     4.892902e+04   
min         0.000000         0.000000   0.000000e+00     0.000000e+00   
25%       121.000000       115.000000   6.500000e+01     7.700000e+01   
50%       325.000000       300.000000   2.040000e+02     2.320000e+02   
75%      1008.750000       971.750000   9.527500e+02     1.083000e+03   
max    188264.000000    185298.000000   1.529785e+06     1.731856e+06   

       NA_MALE_TOTAL  NA_FEMALE_TOTAL  
count    3142.000000      3142.000000  
mean      387.793125       382.213558  
std      3681.779723      3625.889024  
min         0.000000         0.000000  
25%         9.000000         8.000000  
50%        28.000000        25.000000  
75%       108.750000       100.000000  
max    171165.000000    166656.000000  

[8 rows x 88 columns]

In [38]:
race_current.rename(columns={'CTYNAME': 'County Name'}, inplace=True)
race_current.rename(columns={'STATE': 'stateFIPS'}, inplace=True)
race_current.rename(columns={'COUNTY': 'countyFIPS_2d'}, inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [39]:
race_current['countyFIPS_2d'] = race_current['countyFIPS_2d'].astype(str).str.zfill(3)
race_current['stateFIPS'] = race_current['stateFIPS'].astype(str).str.zfill(2)
race_current["countyFIPS"] = race_current["stateFIPS"] + race_current["countyFIPS_2d"]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [40]:
test_race_sd = race_current.loc[(race_current["countyFIPS"] == "46102")]
test_race_sd

SUMLEV stateFIPS countyFIPS_2d        STNAME           County Name  \
550145      50        46           102  South Dakota  Oglala Lakota County   

        YEAR  AGEGRP  TOT_POP  TOT_MALE  TOT_FEMALE  ...  WA_FEMALE_TOTAL  \
550145    12       0    14177      6961        7216  ...              844   

        BA_MALE_TOTAL  BA_FEMALE_TOTAL  IA_MALE_TOTAL  IA_FEMALE_TOTAL  \
550145             94               69          12904            13475   

        AA_MALE_TOTAL  AA_FEMALE_TOTAL  NA_MALE_TOTAL  NA_FEMALE_TOTAL  \
550145             29               37             12               13   

        countyFIPS  
550145       46102  

[1 rows x 91 columns]

In [41]:
race_clean = race_current.loc[:,["countyFIPS", "stateFIPS", "countyFIPS_2d", "STNAME", "County Name", "TOT_POP", "WA_MALE_TOTAL", "WA_FEMALE_TOTAL"
               , "BA_MALE_TOTAL", "BA_FEMALE_TOTAL", "IA_MALE_TOTAL", "IA_FEMALE_TOTAL"
               , "AA_MALE_TOTAL", "AA_FEMALE_TOTAL", "NA_MALE_TOTAL", "NA_FEMALE_TOTAL"]]

In [42]:
null_data_race = race_clean[race_clean.isnull().any(axis=1)]
null_data_race

Empty DataFrame
Columns: [countyFIPS, stateFIPS, countyFIPS_2d, STNAME, County Name, TOT_POP, WA_MALE_TOTAL, WA_FEMALE_TOTAL, BA_MALE_TOTAL, BA_FEMALE_TOTAL, IA_MALE_TOTAL, IA_FEMALE_TOTAL, AA_MALE_TOTAL, AA_FEMALE_TOTAL, NA_MALE_TOTAL, NA_FEMALE_TOTAL]
Index: []

In [43]:
test_race = race_clean.loc[(race_clean["countyFIPS"] == "46102")]
test_race

countyFIPS stateFIPS countyFIPS_2d        STNAME           County Name  \
550145      46102        46           102  South Dakota  Oglala Lakota County   

        TOT_POP  WA_MALE_TOTAL  WA_FEMALE_TOTAL  BA_MALE_TOTAL  \
550145    14177            889              844             94   

        BA_FEMALE_TOTAL  IA_MALE_TOTAL  IA_FEMALE_TOTAL  AA_MALE_TOTAL  \
550145               69          12904            13475             29   

        AA_FEMALE_TOTAL  NA_MALE_TOTAL  NA_FEMALE_TOTAL  
550145               37             12               13

In [44]:
race_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3142 entries, 209 to 716357
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   countyFIPS       3142 non-null   object
 1   stateFIPS        3142 non-null   object
 2   countyFIPS_2d    3142 non-null   object
 3   STNAME           3142 non-null   object
 4   County Name      3142 non-null   object
 5   TOT_POP          3142 non-null   int64 
 6   WA_MALE_TOTAL    3142 non-null   int64 
 7   WA_FEMALE_TOTAL  3142 non-null   int64 
 8   BA_MALE_TOTAL    3142 non-null   int64 
 9   BA_FEMALE_TOTAL  3142 non-null   int64 
 10  IA_MALE_TOTAL    3142 non-null   int64 
 11  IA_FEMALE_TOTAL  3142 non-null   int64 
 12  AA_MALE_TOTAL    3142 non-null   int64 
 13  AA_FEMALE_TOTAL  3142 non-null   int64 
 14  NA_MALE_TOTAL    3142 non-null   int64 
 15  NA_FEMALE_TOTAL  3142 non-null   int64 
dtypes: int64(11), object(5)
memory usage: 417.3+ KB


### Incidence of Pre-existing Conditions & Coverage of Flu Vaccine

People of any age with the following conditions are at increased risk of severe illness from COVID-19 (according to CDC, 17 July 17 2020:

PolicyMap worked with journalists at the New York Times to create this index assessing a county’s relative risk of its population developing severe COVID-19 symptoms. The index represents the relative risk for a high proportion of residents in each county to develop serious health complications from COVID-19 because of underlying health conditions identified by the CDC as contributing to a person’s risk of developing severe symptoms from the virus. These conditions include COPD, heart disease, high blood pressure, diabetes, and obesity.

Estimates of COPD, heart disease, high blood pressure, and diabetes and obesity prevalence at the tract and ZCTA level are from PolicyMap’s Health Outcome Estimates. Estimates of diabetes and obesity prevalence at the county level are from the CDC’s U.S. Diabetes Surveillance System.

Normalized scores were then converted to percentiles and z scores for easier interpretation. Percentiles rank counties from the lowest score to the highest on a scale of 0 to 100, where a score of 50 represents the median value. A county’s z score shows how many standard deviations above or below the average a county’s risk level falls. A score of 0.6, for example, would mean that the county has a higher risk than average, but is still within one standard deviation of the average and is therefore not unusually high. Risk categories from very low to very high are assigned based on z scores.

Constrained features to the following (according to CDC advisory 28 July, 2020):
- Serious heart conditions, such as heart failure, coronary artery disease, or cardiomyopathies (CVDINFR4, CVDCRHD4)
- Cancer (CHCOCNCR)
- Chronic kidney disease (CHCKDNY)
- COPD (CHCCOPD1)
- Obesity (BMI> 30) ( _BMI5CAT value 4; not available at county level)
- Sickle cell disease (not available)
- Solid organ transplantation 
- Type 2 diabetes mellitus (proxy; taking insulin: INSULIN)


Proxy Prevention Coverage
- Adult flu shot/spray past 12 mos (FLUSHOT6)


REFERENCES:
1. Covid 19 People with Certain Medical Conditions https://www.cdc.gov/coronavirus/2019-ncov/need-extra-precautions/people-with-medical-conditions.html?CDC_AA_refVal=https%3A%2F%2Fwww.cdc.gov%2Fcoronavirus%2F2019-ncov%2Fneed-extra-precautions%2Fgroups-at-higher-risk.html
2. Centers for Disease Control and Prevention (CDC). Behavioral Risk Factor Surveillance System Survey Data. Atlanta, Georgia: U.S. Department of Health and Human Services, Centers for Disease Control and Prevention, 2017.: https://www.cdc.gov/brfss/smart/smart_2017.html
3. Evidence used to update the list of underlying medical conditions that increase a person’s risk of severe illness from COVID-19: https://www.cdc.gov/coronavirus/2019-ncov/need-extra-precautions/evidence-table.html
4. PolicyMap Severe COVID-19 Health Risk Index: https://www.policymap.com/download-covid19-data/

In [45]:
# CDC SMART Data
# preexisting = pd.read_sas("data/llcp2018_2.xpt")
# preexisting.to_csv('data/llcp2018.csv')
# preexisting = pd.read_csv("data/MMSA2017.csv")
# preexisting["_MMSA"] = preexisting["_MMSA"].astype(str)
# print(preexisting.dtypes)
# preexisting['countyFIPS_2d'] = preexisting['_MMSA'].str[2:4]
# preexisting['stateFIPS_2d'] = preexisting['_MMSA'].str[0:2]

In [46]:
preexisting = pd.read_csv("data/risk_clean6.csv")

In [47]:
preexisting

countyFIPS  stateFIPS  countyFIPS_2d  index_percentile index_category
0           1001          1              1             65.42  Above Average
1           1003          1              3             68.39  Above Average
2           1005          1              5             97.09           High
3           1007          1              7             83.36  Above Average
4           1009          1              9             81.75  Above Average
...          ...        ...            ...               ...            ...
3138       56037         56             37             10.42            Low
3139       56039         56             39              2.94       Very low
3140       56041         56             41             27.13  Below Average
3141       56043         56             43             32.76  Below Average
3142       56045         56             45             33.16  Below Average

[3143 rows x 5 columns]

In [48]:
print(preexisting.dtypes)

countyFIPS            int64
stateFIPS             int64
countyFIPS_2d         int64
index_percentile    float64
index_category       object
dtype: object


In [49]:
preexisting.describe()

countyFIPS    stateFIPS  countyFIPS_2d  index_percentile
count   3143.000000  3143.000000    3143.000000       3143.000000
mean   30390.372574    30.286669     103.703786         48.120646
std    15164.780323    15.146439     107.936692        181.451891
min     1001.000000     1.000000       1.000000      -9999.000000
25%    18178.000000    18.000000      35.000000         27.020000
50%    29177.000000    29.000000      79.000000         51.380000
75%    45082.000000    45.000000     133.000000         75.470000
max    56045.000000    56.000000     840.000000        100.000000

In [50]:
preexisting.rename(columns={'index_percentile': 'Risk Index'}, inplace=True)

In [51]:
preexisting["countyFIPS"] = preexisting["countyFIPS"].astype(str).str.zfill(5)
preexisting["stateFIPS"] = preexisting["stateFIPS"].astype(str).str.zfill(2)
preexisting["countyFIPS_2d"] = preexisting["countyFIPS_2d"].astype(str).str.zfill(3)

# preexisting["Risk Index"] = preexisting["Risk Index"].astype(str)

In [52]:
null_data_preexisting = preexisting[preexisting.isnull().any(axis=1)]
null_data_preexisting

countyFIPS stateFIPS countyFIPS_2d  Risk Index index_category
2916      51515        51           515     -9999.0            NaN

In [53]:
preexisting_clean = preexisting.dropna()

In [54]:
null_data_preexisting = preexisting_clean[preexisting_clean.isnull().any(axis=1)]
null_data_preexisting

Empty DataFrame
Columns: [countyFIPS, stateFIPS, countyFIPS_2d, Risk Index, index_category]
Index: []

In [55]:
test_preexisting = preexisting_clean.loc[(preexisting_clean["countyFIPS"] == "1001")]
test_preexisting

Empty DataFrame
Columns: [countyFIPS, stateFIPS, countyFIPS_2d, Risk Index, index_category]
Index: []

In [56]:
test_preexisting.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   countyFIPS      0 non-null      object 
 1   stateFIPS       0 non-null      object 
 2   countyFIPS_2d   0 non-null      object 
 3   Risk Index      0 non-null      float64
 4   index_category  0 non-null      object 
dtypes: float64(1), object(4)
memory usage: 0.0+ bytes


### Flu Coverage (CDC Wonder)? 

## Merging DataFrames

In [57]:
merge_cases_death = covid_cases_clean.merge(covid_deaths_clean, on=["stateFIPS", "countyFIPS_2d", "countyFIPS"], how='left', validate="1:1")

In [58]:
merge_cases_death.info()
null_data_m_1 = merge_cases_death[merge_cases_death.isnull().any(axis=1)]
null_data_m_1

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3145 entries, 0 to 3144
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   countyFIPS     3145 non-null   object
 1   stateFIPS      3145 non-null   object
 2   countyFIPS_2d  3145 non-null   object
 3   County Name_x  3145 non-null   object
 4   State_x        3145 non-null   object
 5   Total Cases    3145 non-null   int64 
 6   County Name_y  3145 non-null   object
 7   State_y        3145 non-null   object
 8   Total Deaths   3145 non-null   int64 
dtypes: int64(2), object(7)
memory usage: 245.7+ KB


Empty DataFrame
Columns: [countyFIPS, stateFIPS, countyFIPS_2d, County Name_x, State_x, Total Cases, County Name_y, State_y, Total Deaths]
Index: []

In [59]:
merge_cases_death

countyFIPS stateFIPS countyFIPS_2d      County Name_x State_x  \
0         01001        01           001     Autauga County      AL   
1         01003        01           003     Baldwin County      AL   
2         01005        01           005     Barbour County      AL   
3         01007        01           007        Bibb County      AL   
4         01009        01           009      Blount County      AL   
...         ...       ...           ...                ...     ...   
3140      56037        56           037  Sweetwater County      WY   
3141      56039        56           039       Teton County      WY   
3142      56041        56           041       Uinta County      WY   
3143      56043        56           043    Washakie County      WY   
3144      56045        56           045      Weston County      WY   

      Total Cases      County Name_y State_y  Total Deaths  
0           39746     Autauga County      AL           909  
1           76970     Baldwin County      AL           958  
2           24625     Barbour County      AL           155  
3           13636        Bibb County      AL           103  
4           19311      Blount County      AL            82  
...           ...                ...     ...           ...  
3140         7361  Sweetwater County      WY            34  
3141        13823       Teton County      WY           101  
3142         9737       Uinta County      WY             0  
3143         3104    Washakie County      WY           291  
3144          130      Weston County      WY             0  

[3145 rows x 9 columns]

In [60]:
merge_cases_death_pov = merge_cases_death.merge(poverty_clean, on=["stateFIPS", "countyFIPS_2d", "countyFIPS"], how='left', validate="1:1")

In [61]:
merge_cases_death_pov

countyFIPS stateFIPS countyFIPS_2d      County Name_x State_x  \
0         01001        01           001     Autauga County      AL   
1         01003        01           003     Baldwin County      AL   
2         01005        01           005     Barbour County      AL   
3         01007        01           007        Bibb County      AL   
4         01009        01           009      Blount County      AL   
...         ...       ...           ...                ...     ...   
3140      56037        56           037  Sweetwater County      WY   
3141      56039        56           039       Teton County      WY   
3142      56041        56           041       Uinta County      WY   
3143      56043        56           043    Washakie County      WY   
3144      56045        56           045      Weston County      WY   

      Total Cases      County Name_y State_y  Total Deaths        County Name  \
0           39746     Autauga County      AL           909     Autauga County   
1           76970     Baldwin County      AL           958     Baldwin County   
2           24625     Barbour County      AL           155     Barbour County   
3           13636        Bibb County      AL           103        Bibb County   
4           19311      Blount County      AL            82      Blount County   
...           ...                ...     ...           ...                ...   
3140         7361  Sweetwater County      WY            34  Sweetwater County   
3141        13823       Teton County      WY           101       Teton County   
3142         9737       Uinta County      WY             0       Uinta County   
3143         3104    Washakie County      WY           291    Washakie County   
3144          130      Weston County      WY             0      Weston County   

     State  Poverty Universe, All Ages  
0       AL                     55073.0  
1       AL                    215255.0  
2       AL                     21979.0  
3       AL                     20212.0  
4       AL                     57238.0  
...    ...                         ...  
3140    WY                     42205.0  
3141    WY                     22888.0  
3142    WY                     20135.0  
3143    WY                      7735.0  
3144    WY                      6595.0  

[3145 rows x 12 columns]

In [62]:
test_data_m_2 = merge_cases_death_pov.loc[(merge_cases_death_pov["countyFIPS"] == "02270")]
test_data_m_2

countyFIPS stateFIPS countyFIPS_2d             County Name_x State_x  \
93      02270        02           270  Wade Hampton Census Area      AK   

    Total Cases             County Name_y State_y  Total Deaths  \
93            0  Wade Hampton Census Area      AK             0   

                 County Name State  Poverty Universe, All Ages  
93  Wade Hampton Census Area    AK               299891.036329

In [63]:
merge_cases_death_pov.info()

null_data_m_2 = merge_cases_death_pov[merge_cases_death_pov.isnull().any(axis=1)]
null_data_m_2

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3145 entries, 0 to 3144
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   countyFIPS                  3145 non-null   object 
 1   stateFIPS                   3145 non-null   object 
 2   countyFIPS_2d               3145 non-null   object 
 3   County Name_x               3145 non-null   object 
 4   State_x                     3145 non-null   object 
 5   Total Cases                 3145 non-null   int64  
 6   County Name_y               3145 non-null   object 
 7   State_y                     3145 non-null   object 
 8   Total Deaths                3145 non-null   int64  
 9   County Name                 3144 non-null   object 
 10  State                       3144 non-null   object 
 11  Poverty Universe, All Ages  3144 non-null   float64
dtypes: float64(1), int64(2), object(9)
memory usage: 319.4+ KB


countyFIPS stateFIPS countyFIPS_2d                       County Name_x  \
1830      00001        36           001  New York City Unallocated/Probable   

     State_x  Total Cases                       County Name_y State_y  \
1830      NY          739  New York City Unallocated/Probable      NY   

      Total Deaths County Name State  Poverty Universe, All Ages  
1830         79004         NaN   NaN                         NaN

In [64]:
merge_cases_death_pov_2 = merge_cases_death_pov.dropna()

In [65]:
null_data_m_3 = merge_cases_death_pov_2[merge_cases_death_pov_2.isnull().any(axis=1)]
null_data_m_3

Empty DataFrame
Columns: [countyFIPS, stateFIPS, countyFIPS_2d, County Name_x, State_x, Total Cases, County Name_y, State_y, Total Deaths, County Name, State, Poverty Universe, All Ages]
Index: []

In [66]:
merge_cases_death_pov_race = merge_cases_death_pov_2.merge(race_clean, on=[
    "stateFIPS", "countyFIPS_2d", "countyFIPS"], how='left', validate="1:1")

In [67]:
merge_cases_death_pov_race["countyFIPS"] = merge_cases_death_pov_race["countyFIPS"].astype(str).str.zfill(5)
merge_cases_death_pov_race.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3144 entries, 0 to 3143
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   countyFIPS                  3144 non-null   object 
 1   stateFIPS                   3144 non-null   object 
 2   countyFIPS_2d               3144 non-null   object 
 3   County Name_x               3144 non-null   object 
 4   State_x                     3144 non-null   object 
 5   Total Cases                 3144 non-null   int64  
 6   County Name_y               3144 non-null   object 
 7   State_y                     3144 non-null   object 
 8   Total Deaths                3144 non-null   int64  
 9   County Name_x               3144 non-null   object 
 10  State                       3144 non-null   object 
 11  Poverty Universe, All Ages  3144 non-null   float64
 12  STNAME                      3142 non-null   object 
 13  County Name_y               3142 

In [68]:
merge_cases_death_pov_race.info()

null_data_m_4 = merge_cases_death_pov_race[merge_cases_death_pov_race.isnull().any(axis=1)]
null_data_m_4

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3144 entries, 0 to 3143
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   countyFIPS                  3144 non-null   object 
 1   stateFIPS                   3144 non-null   object 
 2   countyFIPS_2d               3144 non-null   object 
 3   County Name_x               3144 non-null   object 
 4   State_x                     3144 non-null   object 
 5   Total Cases                 3144 non-null   int64  
 6   County Name_y               3144 non-null   object 
 7   State_y                     3144 non-null   object 
 8   Total Deaths                3144 non-null   int64  
 9   County Name_x               3144 non-null   object 
 10  State                       3144 non-null   object 
 11  Poverty Universe, All Ages  3144 non-null   float64
 12  STNAME                      3142 non-null   object 
 13  County Name_y               3142 

countyFIPS stateFIPS countyFIPS_2d               County Name_x State_x  \
93       02270        02           270    Wade Hampton Census Area      AK   
187      06000        06           000  Grand Princess Cruise Ship      CA   

     Total Cases               County Name_y State_y  Total Deaths  \
93             0    Wade Hampton Census Area      AK             0   
187         3087  Grand Princess Cruise Ship      CA             0   

                County Name_x  ... WA_MALE_TOTAL  WA_FEMALE_TOTAL  \
93   Wade Hampton Census Area  ...           NaN              NaN   
187                California  ...           NaN              NaN   

    BA_MALE_TOTAL BA_FEMALE_TOTAL  IA_MALE_TOTAL  IA_FEMALE_TOTAL  \
93            NaN             NaN            NaN              NaN   
187           NaN             NaN            NaN              NaN   

     AA_MALE_TOTAL  AA_FEMALE_TOTAL  NA_MALE_TOTAL  NA_FEMALE_TOTAL  
93             NaN              NaN            NaN              NaN  
187            NaN              NaN            NaN              NaN  

[2 rows x 25 columns]

In [69]:
merge_cases_death_pov_race_2 = merge_cases_death_pov_race.dropna()


test_data_m_5 = merge_cases_death_pov_race_2.loc[(merge_cases_death_pov_race_2["countyFIPS"] == "06000")]
test_data_m_5

Empty DataFrame
Columns: [countyFIPS, stateFIPS, countyFIPS_2d, County Name_x, State_x, Total Cases, County Name_y, State_y, Total Deaths, County Name_x, State, Poverty Universe, All Ages, STNAME, County Name_y, TOT_POP, WA_MALE_TOTAL, WA_FEMALE_TOTAL, BA_MALE_TOTAL, BA_FEMALE_TOTAL, IA_MALE_TOTAL, IA_FEMALE_TOTAL, AA_MALE_TOTAL, AA_FEMALE_TOTAL, NA_MALE_TOTAL, NA_FEMALE_TOTAL]
Index: []

[0 rows x 25 columns]

In [71]:
# merge_cases_death_pov_race_2["countyFIPS"] = merge_cases_death_pov_race_2["countyFIPS"].astype(str).str.zfill(5)
merge_cases_death_pov_race_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3142 entries, 0 to 3143
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   countyFIPS                  3142 non-null   object 
 1   stateFIPS                   3142 non-null   object 
 2   countyFIPS_2d               3142 non-null   object 
 3   County Name_x               3142 non-null   object 
 4   State_x                     3142 non-null   object 
 5   Total Cases                 3142 non-null   int64  
 6   County Name_y               3142 non-null   object 
 7   State_y                     3142 non-null   object 
 8   Total Deaths                3142 non-null   int64  
 9   County Name_x               3142 non-null   object 
 10  State                       3142 non-null   object 
 11  Poverty Universe, All Ages  3142 non-null   float64
 12  STNAME                      3142 non-null   object 
 13  County Name_y               3142 

In [72]:
merge_cases_death_pov_race_2 = merge_cases_death_pov_race_2.merge(preexisting_clean, on=["stateFIPS", "countyFIPS_2d", "countyFIPS"], how='left', validate="1:1")

In [73]:
merge_cases_death_pov_race_2

countyFIPS stateFIPS countyFIPS_2d      County Name_x State_x  \
0         01001        01           001     Autauga County      AL   
1         01003        01           003     Baldwin County      AL   
2         01005        01           005     Barbour County      AL   
3         01007        01           007        Bibb County      AL   
4         01009        01           009      Blount County      AL   
...         ...       ...           ...                ...     ...   
3137      56037        56           037  Sweetwater County      WY   
3138      56039        56           039       Teton County      WY   
3139      56041        56           041       Uinta County      WY   
3140      56043        56           043    Washakie County      WY   
3141      56045        56           045      Weston County      WY   

      Total Cases      County Name_y State_y  Total Deaths      County Name_x  \
0           39746     Autauga County      AL           909     Autauga County   
1           76970     Baldwin County      AL           958     Baldwin County   
2           24625     Barbour County      AL           155     Barbour County   
3           13636        Bibb County      AL           103        Bibb County   
4           19311      Blount County      AL            82      Blount County   
...           ...                ...     ...           ...                ...   
3137         7361  Sweetwater County      WY            34  Sweetwater County   
3138        13823       Teton County      WY           101       Teton County   
3139         9737       Uinta County      WY             0       Uinta County   
3140         3104    Washakie County      WY           291    Washakie County   
3141          130      Weston County      WY             0      Weston County   

      ... BA_MALE_TOTAL  BA_FEMALE_TOTAL IA_MALE_TOTAL IA_FEMALE_TOTAL  \
0     ...       10751.0          12270.0         395.0           446.0   
1     ...       19832.0          21115.0        2721.0          2624.0   
2     ...       12743.0          11280.0         285.0           182.0   
3     ...        5951.0           3719.0         159.0           151.0   
4     ...        1174.0           1080.0         592.0           598.0   
...   ...           ...              ...           ...             ...   
3137  ...         828.0            640.0         881.0           801.0   
3138  ...         248.0            181.0         310.0           272.0   
3139  ...         199.0            186.0         391.0           402.0   
3140  ...          80.0             57.0         169.0           198.0   
3141  ...          87.0             60.0         211.0           155.0   

      AA_MALE_TOTAL  AA_FEMALE_TOTAL  NA_MALE_TOTAL  NA_FEMALE_TOTAL  \
0             727.0            879.0           87.0             75.0   
1            2337.0           3394.0          254.0            267.0   
2             127.0            141.0           72.0             41.0   
3              72.0             73.0           50.0             16.0   
4             232.0            265.0          102.0             62.0   
...             ...              ...            ...              ...   
3137          516.0            619.0           97.0             99.0   
3138          358.0            573.0           63.0             43.0   
3139          123.0            180.0           54.0             44.0   
3140           66.0            102.0           13.0             13.0   
3141           78.0            178.0            8.0              2.0   

      Risk Index  index_category  
0          65.42   Above Average  
1          68.39   Above Average  
2          97.09            High  
3          83.36   Above Average  
4          81.75   Above Average  
...          ...             ...  
3137       10.42             Low  
3138        2.94        Very low  
3139       27.13   Below Average  
3140       32.76   Below Average  
3141       33.16   Below Average  

[3142 rows x 27 

In [74]:
null_data_m_6 = merge_cases_death_pov_race_2[merge_cases_death_pov_race_2.isnull().any(axis=1)]
null_data_m_6

Empty DataFrame
Columns: [countyFIPS, stateFIPS, countyFIPS_2d, County Name_x, State_x, Total Cases, County Name_y, State_y, Total Deaths, County Name_x, State, Poverty Universe, All Ages, STNAME, County Name_y, TOT_POP, WA_MALE_TOTAL, WA_FEMALE_TOTAL, BA_MALE_TOTAL, BA_FEMALE_TOTAL, IA_MALE_TOTAL, IA_FEMALE_TOTAL, AA_MALE_TOTAL, AA_FEMALE_TOTAL, NA_MALE_TOTAL, NA_FEMALE_TOTAL, Risk Index, index_category]
Index: []

[0 rows x 27 columns]